<img src="https://spaceplace.nasa.gov/satellite/en/geostationary-orbit.en.gif" alt="From Earth, a geostationary satellite looks like it is always in the same place, because it moves in the same direction and at the same rate the Earth spins. Image credit: NASA Solar System Exploration"></img>

# Coordinate Types

---

## Overview

Great circles use different types of coordinates when working with unit spheres and ellipsoids. This notebook will cover the different types of coordinates and how to convert between them.

1. Types of Coordinates
1. Convert Coordinates to All Coordinate Types
1. Plot Coordinates on a World Map

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Numpy](https://foundations.projectpythia.org/core/numpy.html) | Necessary | Used to work with large arrays |
| [Pandas](https://foundations.projectpythia.org/core/numpy.html) | Necessary | Used to read in and organize data (in particular dataframes) |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | Will be used for adding maps to plotting |
| [Matplotlib](https://foundations.projectpythia.org/core/matplotlib.html) | Helpful | Will be used for plotting |

- **Time to learn**: 20 minutes

---

## Imports

In [ ]:
import numpy as np                                     # working with degrees and radians

import matplotlib.pyplot as plt                        # plotting a graph
from cartopy import crs as ccrs, feature as cfeature   # plotting a world map

## Types of Coordinates

### Geodesic Coordinates

Geodesic coordinates are latitude and longtiude and are measured from -90° South to 90° North and -180° East to 180° West measured from Greenwich.

<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/FedStats_Lat_long.svg/1024px-FedStats_Lat_long.svg.png" alt="Longitude lines are perpendicular to and latitude lines are parallel to the Equator from Wikipedia" width=400 />
</p>

### Cartesian Coordinates

Cartesian coordinates describe points in space based on perpendicular axis lines that meet at a single point of origin, where any point's position is described based on the distance to the origin along xyz axis.

<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Coord_system_CA_0.svg/1024px-Coord_system_CA_0.svg.png" alt="A three dimensional Cartesian coordinate system, with origin O and axis lines X, Y and Z, oriented as shown by the arrows. The tick marks on the axes are one length unit apart. The black dot shows the point with coordinates x = 2, y = 3, and z = 4, or (2, 3, 4) from Wikipedia" width=400 />
</p>

Image Source: [Three Dimensional Cartesian Coordinate System](https://en.wikipedia.org/wiki/Cartesian_coordinate_system)

**Geodesic to Cartesian Coordinates**

Assuming the Earth's radius is 6378137 meters then:

$$x = radius * cos(latitude) * cos(longitude)$$
$$y = radius * cos(latitude) * sin(longitude)$$
$$z = radius * sin(latitude)$$

In [ ]:
def cartesian_coordinates(latitude=None, longitude=None):
    earth_radius = 6378137  # meters
    latitude = np.deg2rad(latitude)
    longitude = np.deg2rad(longitude)
    cart_x = earth_radius * np.cos(latitude) * np.cos(longitude)
    cart_y = earth_radius * np.cos(latitude) * np.sin(longitude)
    cart_z = earth_radius * np.sin(latitude)
    return cart_x, cart_y, cart_z

### Spherical Coordinates

Spherical coordinates describe points in space based on three values: radial distance (rho, r) along the radial line between point and the origin, polar angle (theta, θ) between the radial line and the polar axis, and azimuth angle (phi, φ) which is the angle of rotation of the radial line around the polar axis. With a fixed radius, the 3-point coordinates (r, θ, φ) provide a coordinate along a sphere.

- Radial distance: distance from center to surface of sphere
- Polar angle: angle between radial line and polar axis
- Azimuth angle: angle around polar axis

<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Sphericalcoordinates.svg/1024px-Sphericalcoordinates.svg.png" alt="Spherical Coordinate Description from Wikipedia" width=400 />
</p>

Image Source: [Wikipedia - Spherical Coordinate System](https://simple.wikipedia.org/wiki/Spherical_coordinate_system)

Convert from cartesian (rectangular) coordinates spherical coordinates

$$ρ^2 = x^2 + y^2 + z^2$$
$$tan(θ) = \frac{y}{x}$$
$$φ = arccos(\frac{x}{\sqrt{x^2 + y^2 + z^2}})$$

Where, rho (ρ), theta (θ), phi (φ):

$$ρ = \sqrt{x^2 + y^2 + z^2}$$
$$θ = arctan(\frac{y}{x})$$
$$φ = arccos(\frac{x}{ρ})$$

In [ ]:
def cartesian_to_spherical_coordinates(cart_x=None, cart_y=None, cart_z=None):
    rho = np.sqrt(cart_x**2 + cart_y**2 + cart_z**2)
    theta = np.arctan(cart_y/cart_x)
    phi = np.arccos(cart_z / rho)
    return rho, theta, phi 

### Polar Coordinates

Polar coordinates are a combination of latitude, longitude, and altitude from the center of the sphere (based on the radius).

Assuming the Earth's radius is 6378137 meters then:

$$x = cos(latitude) * cos(longitude) * radius$$
$$y = cos(latitude) * sin(longitude) * radius$$
$$z = sin(latitude) * radius$$

In [ ]:
def polar_coordinates(latitude=None, longitude=None):
    earth_radius = 6378137  # meters
    latitude = np.deg2rad(latitude)
    longitude = np.deg2rad(longitude)
    polar_x = np.cos(latitude) * np.sin(longitude) * earth_radius
    polar_y = np.cos(latitude) * np.cos(longitude) * earth_radius
    polar_z = np.sin(latitude) * earth_radius
    return polar_x, polar_y, polar_z

## Convert City Coordinates to All Coordinate Types

### Display Coordinates of Cities

First, we will read in the latitude and longitude coordinates from locations csv:

In [ ]:
import pandas as pd

location_df = pd.read_csv("../location_coords.txt")
location_df = location_df.rename(columns=lambda x: x.strip()) # strip excess white space from column names and values
location_df

### Add Columns for Additional Coordinate Types

In [ ]:
location_df["cart_x"], location_df["cart_y"], location_df["cart_z"] = cartesian_coordinates(location_df["latitude"],
                                                                                            location_df["longitude"])
location_df["rho"], location_df["theta"], location_df["phi"] = cartesian_to_spherical_coordinates(location_df["cart_x"],
                                                                                                  location_df["cart_y"],
                                                                                                  location_df["cart_z"])
location_df["polar_x"], location_df["polar_y"], location_df["polar_z"] = polar_coordinates(location_df["latitude"],
                                                                                           location_df["longitude"])

location_df

In [ ]:
# Save Output to a New Text File
location_df.to_csv("../location_full_coords.txt", index=False)

## Plot Coordinates

### World Map

Full world map from -180-180 and -90-90:
```
longitude east = 180

longitude west = -180

latitude north = 90

latitude south = -90
```

In [ ]:
# Set up world map plot
fig = plt.subplots(figsize=(15, 10))
projection_map = ccrs.PlateCarree()
ax = plt.axes(projection=projection_map)
lon_west, lon_east, lat_south, lat_north = -180, 180, -90, 90
ax.set_extent([lon_west, lon_east, lat_south, lat_north], crs=projection_map)
ax.coastlines(color="black")
ax.add_feature(cfeature.BORDERS, edgecolor='grey')
ax.add_feature(cfeature.STATES, edgecolor="grey")
    
# Plot Latitude/Longitude Location
longitudes = location_df["longitude"] # longitude
latitudes = location_df["latitude"]   # latitude
plt.scatter(longitudes, latitudes, c="red")

plt.title("World Map with Locations")
plt.show()

### United States Map

Map of the United States roughly from -130 to -60 and 20 to 60:
```
longitude east = -60

longitude west = -130

latitude north = 60

latitude south = 20
```

In [ ]:
# Set up United States map plot
fig = plt.subplots(figsize=(15, 10))
projection_map = ccrs.PlateCarree()
ax = plt.axes(projection=projection_map)
lon_west, lon_east, lat_south, lat_north = -130, -60, 20, 60
ax.set_extent([lon_west, lon_east, lat_south, lat_north], crs=projection_map)
ax.coastlines(color="black")
ax.add_feature(cfeature.BORDERS, edgecolor='grey')
ax.add_feature(cfeature.STATES, edgecolor="grey")
    
# Plot Latitude/Longitude Location
longitudes = location_df["longitude"] # longitude
latitudes = location_df["latitude"]   # latitude
plt.scatter(longitudes, latitudes, c="red")

plt.title("United States Map with Locations")
plt.show()

---

## Summary

Coordinates on the Earth are measured in many different types of coordinate systems: Geodesic (latitude/longitude), cartesian, spherical, and polar. These coordinates will make future calculations simpler by converting a 2D coordinate like latitude/longitude into a 3D space that can be used for vector calculations.

In Python, coordinates can be mapped on to a world map via `matplotlib` and `cartopy`.

### What's next?

Great Circle arcs and paths


## Resources and references


- [Ed Williams "Avitation Formulary"](https://edwilliams.org/avform147.htm)
- [Movable Type Scripts](https://www.movable-type.co.uk/scripts/latlong.html)